# Project 1 - IR CIn UFPE 2022.1

Author:

Matheus Rodrigues de Souza Félix (matheusrdgsf@gmail.com) \
Rodrigo Melo

In [38]:
!pip install git-python

### Libs

In [57]:
import os
import pandas as pd
from git import Repo
import gzip
from io import BytesIO
import gzip

### Data Collect

In [91]:
DATA_PATH = "dataset/scifact-generated-queries"

In [93]:
if not os.path.isdir(DATA_PATH):
    Repo.clone_from("https://huggingface.co/datasets/BeIR/scifact-generated-queries", "dataset/scifact-generated-queries")

In [101]:
with gzip.open(DATA_PATH+"/train.jsonl.gz", 'rb') as f:
    file_content = f.read()

In [108]:
dataset = pd.read_json(BytesIO(file_content), lines=True)

In [114]:
dataset.groupby(["title"]).size().reset_index(name='queries_count')

,title,queries_count
0,PHENIX: a comprehensive Python...,3
1,DNA methylation and healthy human aging,3
2,US-SOMO HPLC-SAXS module: dealing with capill...,3
3,"""Grazing"": a high-risk behavior.",3
4,"""Natural"" killer cells in the mouse. I. Cytoto...",3
...,...,...
5176,β-catenin-independent WNT signaling in basal-l...,3
5177,β-site amyloid precursor protein-cleaving enzy...,3
5178,β1 integrin mediates an alternative survival p...,3
5179,‘Short stature in children - a questionnaire f...,3


## Pre-rocesing Data

In this step we will create 5 data models:

v1 - Only Tokenization; \
v2 - Only Stopword Filter; \
v3 - Only Stemming; \
v4 - Remove Stopwords and Stemming; \
v5 - Remove Stopworpd and expand words with Synonyms.


fatal: cannot mkdir https://github.com/MatheusRDG/information-retrieval-cin-2022: Invalid argument


### Vectorizing Data

### Evaluation